In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Original Architecture

In [4]:
# Generator
class Generator(nn.Module):
    def __init__(self, nz=3):  # nz: latent vector size
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Layer 1: 1x1x3 -> 2x2x1
            nn.ConvTranspose2d(nz, 1, kernel_size=2, stride=2, padding=0, bias=0.1),
            nn.BatchNorm2d(1),
            nn.ReLU(True),
            # Layer 2: 2x2x1 -> 4x4x1
            nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, padding=0, bias=0.1),
            nn.BatchNorm2d(1),
            nn.ReLU(True),
            # Layer 3: 4x4x1 -> 8x8x1
            nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, padding=0, bias=0.1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

In [5]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Layer 1: 8x8x1 -> 4x4x1
            nn.Conv2d(1, 1, kernel_size=2, stride=2, padding=0, bias=0.1),
            nn.ReLU(True),
            # Layer 2: 4x4x1 -> 2x2x1
            nn.Conv2d(1, 1, kernel_size=2, stride=2, padding=0, bias=0.1),
            nn.BatchNorm2d(1),
            nn.ReLU(True)
        )
        # Final layer: 2x2x1 -> 1
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1 * 2 * 2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.main(x)
        print(x.shape)
        return self.fc(x)

# Architecture with detail print

In [17]:
class Generator(nn.Module):
    def __init__(self, nz=3):
        super(Generator, self).__init__()
        self.layer1 = nn.ConvTranspose2d(nz, 1, kernel_size=2, stride=2, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(1)
        self.layer2 = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(1)
        self.layer3 = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, padding=0, bias=False)
        self.tanh = nn.Tanh()

    def forward(self, x):
        print(f"Input Shape: {x.shape}")
        print(f"Input: {x}")

        x = self.layer1(x)
        print(f"Layer1 kernel: {self.layer1.weight.data}, Output: {x}, Shape: {x.shape}")
        x = self.bn1(x)
        x = F.relu(x)

        x = self.layer2(x)
        x = self.bn2(x)
        x = F.relu(x)

        x = self.layer3(x)
        x = self.tanh(x)

        return x

In [18]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.layer1 = nn.Conv2d(1, 1, kernel_size=2, stride=2, padding=0, bias=False)
        self.layer2 = nn.Conv2d(1, 1, kernel_size=2, stride=2, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(1 * 2 * 2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        print(f"Input: {x.shape}")

        x = self.layer1(x)
        x = F.relu(x)

        x = self.layer2(x)
        print(f"Layer2 kernel: {self.layer2.weight.data}, Output: {x}:.2f")
        x = self.bn2(x)
        print(f"BN2 Output: {x}")
        x = F.relu(x)

        x = self.flatten(x)
        print(f"Flattened: {x.shape}")

        x = self.fc(x)
        print(f"FC Output: {x.shape}")
        x = self.sigmoid(x)
        return x

In [22]:
# Initialize models
generator = Generator()
discriminator = Discriminator()

In [26]:
weight_tensor = torch.tensor([[[[ 0.29, -0.39],
                                [-0.25,  0.15]]],
                             [[[ 0.11, -0.13],
                               [ 0.30,  0.34]]],
                             [[[-0.36, -0.27],
                               [ 0.46, -0.17]]]])

generator.layer1.weight.data = weight_tensor
generator.layer1.weight.data

tensor([[[[ 0.2900, -0.3900],
          [-0.2500,  0.1500]]],


        [[[ 0.1100, -0.1300],
          [ 0.3000,  0.3400]]],


        [[[-0.3600, -0.2700],
          [ 0.4600, -0.1700]]]])

In [27]:
z = torch.tensor([[[[0.34]],
                   [[0.13]],
                   [[0.23]]]])
print(z)

tensor([[[[0.3400]],

         [[0.1300]],

         [[0.2300]]]])


In [28]:
# Test generator
fake_img = generator(z)

Input Shape: torch.Size([1, 3, 1, 1])
Input: tensor([[[[0.3400]],

         [[0.1300]],

         [[0.2300]]]])
Layer1 kernel: tensor([[[[ 0.2900, -0.3900],
          [-0.2500,  0.1500]]],


        [[[ 0.1100, -0.1300],
          [ 0.3000,  0.3400]]],


        [[[-0.3600, -0.2700],
          [ 0.4600, -0.1700]]]]), Output: tensor([[[[ 0.0301, -0.2116],
          [ 0.0598,  0.0561]]]], grad_fn=<ConvolutionBackward0>), Shape: torch.Size([1, 1, 2, 2])


In [ ]:
fake_img

tensor([[[[ 0.0000,  0.0000, -0.6639, -0.3132, -0.0463, -0.0188, -0.0463,
           -0.0188],
          [ 0.0000,  0.0000, -0.4090, -0.3230, -0.0251, -0.0194, -0.0251,
           -0.0194],
          [-0.3481, -0.1462,  0.0000,  0.0000, -0.0463, -0.0188, -0.0463,
           -0.0188],
          [-0.1948, -0.1510,  0.0000,  0.0000, -0.0251, -0.0194, -0.0251,
           -0.0194],
          [-0.0463, -0.0188, -0.0463, -0.0188,  0.0000,  0.0000, -0.3980,
           -0.1691],
          [-0.0251, -0.0194, -0.0251, -0.0194,  0.0000,  0.0000, -0.2249,
           -0.1746],
          [-0.0463, -0.0188, -0.0463, -0.0188, -0.2012, -0.0825,  0.0000,
            0.0000],
          [-0.0251, -0.0194, -0.0251, -0.0194, -0.1104, -0.0853,  0.0000,
            0.0000]]]], grad_fn=<TanhBackward0>)

In [30]:
# Test discriminator
img = torch.tensor([[[[-0.01,  0.85, -0.85, -1.40,  0.17, -0.22,  0.71, 0.34],
                      [-0.69,  1.43,  0.23, -1.13,  1.12, -0.28,  0.05, 0.42],
                      [-0.10,  0.91, -0.37, -0.02,  0.03,  0.82, -0.89, 1.48],
                      [ 0.29,  0.18, -0.79,  1.30, -1.02, -1.41, -2.36, -0.29],
                      [-1.58, -1.34,  0.47, -2.29,  0.77, -0.55, -2.16, -1.66],
                      [-2.21, -0.27,  0.26,  0.78, -0.10, -0.56, -0.59, 1.27],
                      [ 0.35,  0.69,  1.20,  1.40, -0.71,  1.41, -1.31, -1.32],
                      [-0.82, -1.65, -0.28, -1.24,  0.74,  0.74, -0.85, 0.04]]]])

In [31]:
concatenated_img = torch.cat((img, fake_img), dim=0)
print(f"Shape of concatenated image: {concatenated_img.shape}")

Shape of concatenated image: torch.Size([2, 1, 8, 8])


In [32]:
disc_output = discriminator(concatenated_img)
print("Discriminator output:", disc_output)

Input: torch.Size([2, 1, 8, 8])
Layer2 kernel: tensor([[[[-0.3388, -0.3432],
          [-0.2917, -0.1711]]]]), Output: tensor([[[[-0.1499, -0.1901],
          [-0.3330, -0.3522]]],


        [[[-0.0726, -0.0665],
          [-0.0925, -0.0901]]]], grad_fn=<ConvolutionBackward0>):.2f
BN2 Output: tensor([[[[ 0.1710, -0.2015],
          [-1.5249, -1.7032]]],


        [[[ 0.8871,  0.9440],
          [ 0.7025,  0.7250]]]], grad_fn=<NativeBatchNormBackward0>)
Flattened: torch.Size([2, 4])
FC Output: torch.Size([2, 1])
Discriminator output: tensor([[0.5221],
        [0.6091]], grad_fn=<SigmoidBackward0>)


In [54]:
# Code lỡ random -> thử batch norm bình thường

In [55]:
features = torch.tensor([[[[-0.1006, -0.2878],
                           [-0.0911, -0.3519]]],
                          [[[-0.3866, -0.0330],
                           [-0.3028, -0.7197]]]])
bn = nn.BatchNorm2d(1)

In [56]:
bn(features)

tensor([[[[ 0.8915, -0.0175],
          [ 0.9376, -0.3288]]],


        [[[-0.4973,  1.2198],
          [-0.0904, -2.1148]]]], grad_fn=<NativeBatchNormBackward0>)